In [2]:
# import pyspark modules
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import Row
from pyspark.sql.types import *       # for datatype conversion
from pyspark.sql.functions import *   # for col() function
from pyspark.ml.linalg import DenseVector
from pyspark.ml.feature import StandardScaler
from pyspark.ml.regression import LinearRegression
import pandas as pd

sc = SparkContext.getOrCreate()
sqlCtx = SQLContext(sc)

# Data Preprocessing

In [4]:
suicide = pd.read_csv ('master.csv')

In [5]:
suicide.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


In [6]:
df = sqlCtx.createDataFrame(suicide)

In [7]:
df.show(5)

+-------+----+------+-----------+-----------+----------+-----------------+------------+------------+------------------+------------------+---------------+
|country|year|   sex|        age|suicides_no|population|suicides/100k pop|country-year|HDI for year| gdp_for_year ($) |gdp_per_capita ($)|     generation|
+-------+----+------+-----------+-----------+----------+-----------------+------------+------------+------------------+------------------+---------------+
|Albania|1987|  male|15-24 years|         21|    312900|             6.71| Albania1987|         NaN|     2,156,624,900|               796|   Generation X|
|Albania|1987|  male|35-54 years|         16|    308000|             5.19| Albania1987|         NaN|     2,156,624,900|               796|         Silent|
|Albania|1987|female|15-24 years|         14|    289700|             4.83| Albania1987|         NaN|     2,156,624,900|               796|   Generation X|
|Albania|1987|  male|  75+ years|          1|     21800|             4

In [8]:
df.count()

27820

In [9]:
df.printSchema()

root
 |-- country: string (nullable = true)
 |-- year: long (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- suicides_no: long (nullable = true)
 |-- population: long (nullable = true)
 |-- suicides/100k pop: double (nullable = true)
 |-- country-year: string (nullable = true)
 |-- HDI for year: double (nullable = true)
 |--  gdp_for_year ($) : string (nullable = true)
 |-- gdp_per_capita ($): long (nullable = true)
 |-- generation: string (nullable = true)



In [10]:
df.cache()

DataFrame[country: string, year: bigint, sex: string, age: string, suicides_no: bigint, population: bigint, suicides/100k pop: double, country-year: string, HDI for year: double,  gdp_for_year ($) : string, gdp_per_capita ($): bigint, generation: string]

In [49]:
#convert sex to indicators
gender ={'male' : 1, 'female' : 0} 
df['sex'] = [gender[item] for item in df["sex"]]      #?

TypeError: Column is not iterable

# Exploratory data analysis

In [25]:
#Describe major attributions
df.describe(['suicides_no','population','suicides/100k pop', 'gdp_per_capita ($)',' gdp_for_year ($) ']).show()
    #suicide/100k pop is optimal than suicides_no
    #problem with the data type of gdp_for_year

+-------+------------------+------------------+------------------+------------------+------------------+
|summary|       suicides_no|        population| suicides/100k pop|gdp_per_capita ($)| gdp_for_year ($) |
+-------+------------------+------------------+------------------+------------------+------------------+
|  count|             27820|             27820|             27820|             27820|             27820|
|   mean|242.57440690150972|1844793.6173975556|12.816097411933846|16866.464414090584|              null|
| stddev| 902.0479168336386|3911779.4417563654|18.961511014503152|18887.576472205557|              null|
|    min|                 0|               278|               0.0|               251| 1,002,219,052,968|
|    max|             22338|          43805214|            224.97|            126352|       997,007,926|
+-------+------------------+------------------+------------------+------------------+------------------+



In [38]:
from pyspark.sql.types import DoubleType
changedTypedf = df.withColumn("changed_gdp_for_year", df[" gdp_for_year ($) "].cast(DoubleType())) 
#df[" gdp_for_year ($) "] = pd.to_numeric(df[" gdp_for_year ($) "])

In [39]:
changedTypedf.describe()

DataFrame[summary: string, country: string, year: string, sex: string, age: string, suicides_no: string, population: string, suicides/100k pop: string, country-year: string, HDI for year: string,  gdp_for_year ($) : string, gdp_per_capita ($): string, generation: string, changed_gdp_for_year: string]

In [29]:
#compute correlation
df.corr('suicides/100k pop','gdp_per_capita ($)')

0.0017851337973438944

In [41]:
# compute correlation between all numeric features
features_numeric = ['suicides_no','population','suicides/100k pop', 'gdp_per_capita ($)']

n_numeric = len(features_numeric)
corr = []

for i in range(0, n_numeric):
 temp = [None] * i

 for j in range(i, n_numeric):
     temp.append(df.corr(features_numeric[i], features_numeric[j]))
 corr.append(temp)

In [42]:
corr
   #suicides_no and population

[[1.0, 0.6161622675219296, 0.3066044512677847, 0.06132974884024606],
 [None, 1.0, 0.008284973053478351, 0.08150985822280572],
 [None, None, 1.0, 0.0017851337973438972],
 [None, None, None, 1.0]]

# Visualization

In [ ]:
#age group for the year span from xxxx to xxxx  -carrie
#world map heat map by year (compare)    -leyi
#group by age group and select a specific country to get a trend plot \time and population  -bryan
#correlation heatmap

In [43]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

## Model

preprocessing:
dummy variable (sex)
missing values(HDI) 
dimensionality reduction PCA

data splitting: 
train 0.8
test 0.2

model building:
--- Supervised Learning
Tree-based methods like random forests and gradient-boosted trees (level of significance) --leyi
Support Vector Machine
Linear Regression (includes feature scaling)  --project3 --carrie

--- Unsupervised Learning
K-mean clustering: Country & Age (generation)  --bryan


model checking:
k-fold cross validation
for regression: MSE
R-square, adjusted R-square
for classification (accuracy, precision, confusion matrix)

what we found:
(answer research question)

Research Questions:
What factor contributes the most to the high suicide rate  of both developed and developing contries and what is the global trend and pattern of the rate of suicidal accident for the past decades?
--age group
--gender
--gdp (developed and developing)
--time span(generation)